In [7]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision

import matplotlib.pyplot as plt
import pandas as pd
import cv2

import numpy as np
import os.path as osp

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
# default `log_dir` is "runs" - 
writer = SummaryWriter('runs/paprika_experiment_1')

In [8]:
labels_file = 'data_new/labels.csv'
root_dir = 'data_new/'

In [9]:
df = pd.read_csv(labels_file)

In [6]:
df.head()

,Id,Directory,File,Complete,Wrinkeled,Cut,Bounce,Hole,Internal Rotting,Sun burn,Broken stamp,Ears,Aphids,Trip
0,4,TripA01,20201019-173127.303.tiff,True,False,False,False,False,False,False,False,False,False,True
1,5,TripA01,20201019-173127.361.tiff,True,False,False,False,False,False,False,False,False,False,True
2,6,TripA01,20201019-173127.419.tiff,True,False,False,False,False,False,False,False,False,False,True
3,7,TripA01,20201019-173127.472.tiff,True,False,False,False,False,False,False,False,False,False,True
4,9,TripA01,20201019-173130.047.tiff,True,False,False,False,False,False,False,False,False,False,True


In [11]:
df.head()

,Id,Directory,File,Complete,Wrinkeled,Cut,Bounce,Hole,Internal Rotting,Sun burn,Broken stamp,Ears,Aphids,Trip,Pitting
0,9,Ingesneden,20201019-171133.256.tiff,True,False,False,False,False,False,False,False,False,False,False,False
1,10,Ingesneden,20201019-171133.802.tiff,True,False,True,False,False,False,False,False,False,False,False,False
2,11,Ingesneden,20201019-171133.862.tiff,True,False,True,False,False,False,False,False,False,False,False,False
3,12,Ingesneden,20201019-171133.915.tiff,True,False,False,False,False,False,False,False,False,False,True,False
4,13,Ingesneden,20201019-171135.154.tiff,True,False,False,False,False,False,True,False,False,False,False,True


In [12]:
df.shape

(1656, 15)

In [ ]:
class PaprikaDataset(Dataset):
    """Paprika dataset."""

    def __init__(self, csv_file, root_dir, label, phase="training", transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.paprika_csv = pd.read_csv(csv_file)
        
        if phase == "training":
            self.paprika_csv = self.paprika_csv.iloc[0:220]
        
        if phase == "validation":
            self.paprika_csv = self.paprika_csv.iloc[220:300]
        
        if phase == "testing":
            self.paprika_csv = self.paprika_csv.iloc[300:]
        
        
        self.root_dir = root_dir
        self.transform = transform
        
        assert label in ['Complete', 'Wrinkeled', 'Cut', 'Bounce', 'Hole', 
                         'Internal Rotting', 'Sun burn', 'Broken stamp', 'Ears',
                         'Aphids', 'Trip'], 'Label is not in columns of CSV file'
        
        self.label = label

    def __len__(self):
        return len(self.paprika_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = osp.join(self.root_dir,
                                self.paprika_csv['Directory'][idx], 
                                self.paprika_csv['File'][idx])
        image = cv2.imread(img_name, -1) # -1 is to read 16-bit
        
        #class_label = np.array(self.paprika_csv[self.label][idx], 'int64')
        class_label = np.array(self.paprika_csv.iloc[idx, 4:], 'float32')
        
        sample = {'image': image, 'label': class_label}
        
        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class Normalize(object):
    """Normalize the input data between [0 1]"""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #image = image.transpose((2, 1, 0))
        image = image[:,:, [2,1,0]]
        imin, imax = np.min(image), np.max(image)
        image -= imin
        imf = np.array(image,'float32')
        imf *= 1./(imax-imin)
        return {'image': imf,
                'label': label}

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 1, 0))
        
        return {'image': torch.from_numpy(image),
                'label': torch.from_numpy(label)}

In [ ]:
paprika_dataset_train = PaprikaDataset(csv_file=labels_file,
                                     root_dir='data/',
                                     label='Trip',   
                                     phase="training",
                                     transform=transforms.Compose([
                                               Normalize(),
                                               ToTensor(),
                                               Rotate(0),
                                               FilpHorizontal(),
                                           ]))

paprika_dataset_val = PaprikaDataset(csv_file=labels_file,
                                     root_dir='data/',
                                     label='Trip',    
                                     phase="validation",
                                     transform=transforms.Compose([
                                               Normalize(),
                                               ToTensor()
                                           ]))

In [ ]:
s= paprika_dataset[0]

In [ ]:
s['image']

In [ ]:
plt.imshow(s['image'].permute(2,1,0))

In [ ]:
def matplotlib_imshow(img):
    plt.imshow(img.permute(2, 1, 0))

In [ ]:
trainloader = DataLoader(paprika_dataset_train, batch_size=16,
                                          shuffle=True, num_workers=4)

valloader = DataLoader(paprika_dataset_val, batch_size=16,
                                          shuffle=True, num_workers=4)

#testloader = DataLoader(testset, batch_size=4,
#                                         shuffle=False, num_workers=2)

In [ ]:
# get some random training images
dataiter = iter(trainloader)
sample = dataiter.next()
images = sample['image']
# create grid of images
img_grid = torchvision.utils.make_grid(images, nrow=4)

# show images
matplotlib_imshow(img_grid)

# write to tensorboard
writer.add_image('16_paprika_images', img_grid)

In [ ]:
class PaprikaNet(nn.Module):
    def __init__(self):
        super(PaprikaNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 7) # input_channels, output_channels, kernel_size
        self.pool = nn.MaxPool2d(5, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.adapmaxpool = nn.AdaptiveAvgPool2d((8,4))
        self.fc1 = nn.Linear(32 * 8 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        x = self.adapmaxpool(x)
        x = x.view(-1, 32 * 8 * 4)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #return x
        return nn.Sigmoid(x)


net = PaprikaNet().to(device)


In [ ]:
s['image'].shape

In [ ]:
s['image'][None, ...].shape

In [ ]:
output = net(s['image'][None, ...].to(device)) # NCHW

In [ ]:
output.shape

In [ ]:
writer.add_graph(net, images.to(device))
writer.close()

In [ ]:
list(net.parameters())

In [ ]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()

optimizer = optim.Adam(net.parameters(), lr=.0001)

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['image'].to(device), data['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    val_loss = validation_loss(net, valloader)
        
    if epoch % 2 == 0:
        #print('Epoch: %d --- loss: %.6f' % (epoch + 1, running_loss / len(trainloader)))
        writer.add_scalar('training loss',
                            running_loss / len(trainloader),
                            epoch)    

print('Finished Training')

In [ ]:
PATH = './paprika_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
def validation_loss(net, valloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data['image'].to(device), data['label'].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return correct/total

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data['image'].to(device), data['label'].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (len(paprika_dataset),
    100 * correct / total))

In [ ]:
[0.4, 0.65, 0.9]

In [ ]:
df=pd.read_csv(labels_file)

In [ ]:
sum(df['Trip'])/320